In [40]:
from aiida import load_profile, orm
from aiida_worktree import node, WorkTree, build_node

load_profile()

Profile<uuid='5ca88b1d2d41436bb1747cc5c08179db' name='default'>

In [41]:
import json

In [42]:
with open("h2o.json","r") as file:
    d = json.load(file)

In [43]:
d

{'workflow': {'functional': 'ki',
  'method': 'dfpt',
  'init_orbitals': 'kohn-sham',
  'from_scratch': True,
  'alpha_numsteps': 1,
  'mt_correction': True,
  'pseudo_library': 'sg15'},
 'atoms': {'cell_parameters': {'vectors': [[9.5142, 0.0, 0.0],
    [0.0, 8.0, 0.0],
    [0.0, 0.0, 8.5861]],
   'units': 'angstrom',
   'periodic': False},
  'atomic_positions': {'units': 'angstrom',
   'positions': [['O', 6.7571, 6.0, 5.9023166667],
    ['H', 7.5142, 6.0, 6.4884166667],
    ['H', 6.0, 6.0, 6.4884166667]]}},
 'calculator_parameters': {'ecutwfc': 45.0, 'ecutrho': 180.0, 'nbnd': 8}}

In [44]:

ndata = {"path": "aiida_koopmans.calculations.kcw.KcwCalculation"}
kcw_calcjob = build_node(ndata)

ndata = {"path": "aiida_quantumespresso.workflows.pw.base.PwBaseWorkChain"}
pw_baseWchain = build_node(ndata)

In [45]:
node = kcw_calcjob()
print("Inputs:")
for input in node.inputs:
    if "." not in input.name:
        print(f"  - {input.name}")
print("Outputs:")
for output in node.outputs:
    if "." not in output.name:
        print(f"  - {output.name}")

Inputs:
  - metadata
  - code
  - monitors
  - remote_folder
  - parameters
  - settings
  - parent_folder
Outputs:
  - remote_folder
  - remote_stash
  - retrieved
  - output_parameters
  - bands


In [46]:
node = pw_baseWchain()
print("Inputs:")
for input in node.inputs:
    if "." not in input.name:
        print(f"  - {input.name}")
print("Outputs:")
for output in node.outputs:
    if "." not in output.name:
        print(f"  - {output.name}")

Inputs:
  - metadata
  - max_iterations
  - clean_workdir
  - handler_overrides
  - pw
  - kpoints
  - kpoints_distance
  - kpoints_force_parity
Outputs:
  - remote_folder
  - remote_stash
  - retrieved
  - output_parameters
  - output_structure
  - output_trajectory
  - output_band
  - output_kpoints
  - output_atomic_occupations


In [47]:
for_ase = {
    "numbers":None,
    "pbc": [False,False,False],
    "positions":d["atoms"]["atomic_positions"]["positions"],
    "cell":d["atoms"]["cell_parameters"]["vectors"]
}

atoms = ""
positions = []
for i in for_ase["positions"]:
    atoms += i[0]
    positions.append(i[1:])


In [48]:
from ase import Atoms
import copy

H2O = Atoms(atoms,
    positions=positions,
    cell=d["atoms"]["cell_parameters"]["vectors"],
    pbc=[0, 0, 0])


In [59]:


structure = orm.StructureData(ase=H2O)

pw_code = orm.load_code("pw-7.2@localhost")
kcw_code = orm.load_code("kcw-7.2@localhost")
pseudo_family = orm.load_group("sg15_v1.2/pbe/SR")
pseudos = pseudo_family.get_pseudos(structure=structure) 
valence = 1+1+6

scf_params = {
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'nbnd': d["calculator_parameters"]["nbnd"],
  'tot_magnetization': 0,
  'nspin': 2,
  'starting_magnetization(1)': 0.0,
  'ecutwfc': d["calculator_parameters"]["ecutwfc"],
  'ecutrho': d["calculator_parameters"]["ecutrho"]},
}

if not d["atoms"]['cell_parameters'].get("periodic", True): scf_params["SYSTEM"]['assume_isolated'] = "mt"

##########################
control_namelist = {
                'kcw_iverbosity':1,
                'kcw_at_ks'      :True,
                'calculation'    :'wann2kcw',
                'lrpa'           :False,
                'mp1'            :1,
                'mp2'            :1,
                'mp3'            :1,
                'homo_only'      :False,
                'read_unitary_matrix' : False,
                'l_vcut'         :False,
                'spin_component' :1,
                }

if not d["atoms"]['cell_parameters'].get("periodic", True): control_namelist['assume_isolated'] = "m-t"

wannier_dict = {
                "check_ks"       : True,
                "num_wann_occ"   : valence,
                "num_wann_emp"   : d["calculator_parameters"]["nbnd"]-valence,
                "have_empty"     : True,
                "has_disentangle": False,
                    }

screening_dict = {
    'tr2'         : 1e-18,
    'nmix'        : 4,
    'niter'       : 33,
    'check_spread': True,
}

ham_dict = {
   'do_bands'       : False,
   'use_ws_distance': True,
   'write_hr'       : True,
   'l_alpha_corr'   : True, #when?
}
##########################


wann2kcw_params = {
        "CONTROL":copy.deepcopy(control_namelist),
        "WANNIER":wannier_dict,
    }

kc_screen_params = {
        "CONTROL":copy.deepcopy(control_namelist),
        "SCREEN":screening_dict,
        "WANNIER":wannier_dict,
    }
kc_screen_params["CONTROL"]["calculation"]="screen"


kc_ham_params = {
        "CONTROL":copy.deepcopy(control_namelist),
        "HAM":ham_dict,
        "WANNIER":wannier_dict,
    }
kc_ham_params["CONTROL"]["calculation"]= "ham"


metadata = {
    'options': {
        'max_wallclock_seconds': 3600,
        'resources': {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 10,
            "num_cores_per_mpiproc": 1,
        },
        'custom_scheduler_commands': u"export OMP_NUM_THREADS="+str(1),
    }
}

In [62]:
from aiida_worktree import WorkTree

wt = WorkTree("scf_kcw_H2O")

# SCF node:
scf = wt.nodes.new(pw_baseWchain, name="scf")
scf.set_from_protocol(
    code = pw_code,
    structure = structure,
    overrides = {"pw":{"parameters":scf_params}}
)
scf.inputs["pw"].value["metadata"]=metadata
scf.inputs["pw"].value["metadata"]=metadata
scf.inputs["pw"].value["pseudos"]=pseudos

# KC conversion node:
kc_conv = wt.nodes.new(kcw_calcjob, name="kc_conversion")
kc_conv.set(
    {
        "code": kcw_code,
        "parameters": orm.Dict(wann2kcw_params),
        "metadata": metadata,
    }
)
wt.links.new(scf.outputs["remote_folder"], kc_conv.inputs["parent_folder"]) # link SCF and KC conversion


# KC screen node:
kc_screen = wt.nodes.new(kcw_calcjob, name="kc_screen")
kc_screen.set(
    {
        "code": kcw_code,
        "parameters": orm.Dict(kc_screen_params),
        "metadata": metadata,
    }
)
wt.links.new(kc_conv.outputs["remote_folder"], kc_screen.inputs["parent_folder"]) # link KC conversion and KC screen


# KC ham node:
kc_ham = wt.nodes.new(kcw_calcjob, name="kc_ham")
kc_ham.set(
    {
        "code": kcw_code,
        "parameters": orm.Dict(kc_ham_params),
        "metadata": metadata,
    }
)
wt.links.new(kc_screen.outputs["remote_folder"], kc_ham.inputs["parent_folder"]) # link KC screen and KC ham

In [63]:
wt.submit(wait=True)

WorkTree node created, PK: 1178


In [ ]:
!verdi process report 1178

: 